## 3. Story Generation with LSTM

We start first by looking at how to generate stories using an LSTM.  To do so we need to do two important things with the text:

1. We need to tokenize the text, converting the words into integers.
2. We need to use an embedding layer before feeding the words to the LSTM.

You may do this lab on Google Colab if you wish.

We begin by first installing our dependencies. Note that xformers must be installed last or there will be dependency breakages.

In [1]:
! pip install --no-cache-dir torch

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
! pip install --no-cache-dir tensorflow transformers datasets numpy scikit-learn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
      --------------------------------------- 0.1/9.3 MB 798.9 kB/s eta 0:00:12
      --------------------------------------- 0.1/9.3 MB 607.9 kB/s eta 0:00:16
      --------------------------------------- 0.2/9.3 MB 573.4 kB/s eta 0:00:17
      --------------------------------------- 0.2/9.3 MB 573.4 kB/s eta 0:00:17
      --------------------------------------- 0.2/9.3 MB 573.4 kB/s eta 0:00:17
      --------------------------------------- 0.2/9.3 MB 573.4 kB/s eta 0:00:17
      --------------------------------------- 0.2/9.3 MB 573.4 kB/s eta 0:00:17
      --------------------------------------- 0.2/9.3 MB 573.4 kB/s eta 0:00:17
      ---------

In [3]:
! pip install --no-cache-dir xformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     - -------------------------------------- 0.1/4.1 MB 297.7 kB/s eta 0:00:14
     - -------------------------------------- 0.1/4.1 MB 297.7 kB/s eta 0:00:14
     - -------------------------------------- 0.1/4.1 MB 297.7 kB/s eta 0:00:14
     - -------------------------------------- 0.2/4.1 MB 316.5 kB/s eta 0:00:13
     - -------------------------------------- 0.2/4

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [234 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\xformers
      copying xformers\attn_bias_utils.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\checkpoint.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\info.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\test.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\utils.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\_cpp_lib.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\_deprecation_warning.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\__init__.py -> build\lib.win-amd64-cpython-312\xformers
      creating build\



### <u>Question 1</u>

Using Google or otherwise, explain i) what an embedding layer does and ii) why we cannot just feed the integers from the tokenizer direct to the LSTM.

<b>Fill your answers in the answer book</b>

Let's begin by creating the dataset. When you unzipped the file containing this lab, it has created for you a text corpus in the sherlock directory, containing several Sherlock Holmes novels in a training directory and a testing directory. 


### 3.1 Loading the Dataset

Keras has its own dataset manipulation libraries, but the one provided by Hugging Face is much more powerful and we will use it. We do the following:

1. Gather all the files in the training and testing directories.
2. Use load_dataset to load up all the texts.
3. Remove all sentences that are too short.
3. Create a special function to convert all the text to lowercase.
4. Tokenize the dataset, converting all the words to integers.
5. Combine the tokens into a single long vector.



In [4]:
# load_dataset from Hugging Face
from datasets import load_dataset

# Search for files in a directory matching a pattern.
import glob

# Gather all the files together
traindir = "sherlock/Train"
testdir = "sherlock/Test"

# Get all the training and testing filenames
train_files = [file for file in glob.glob(traindir + "/*.txt")]
test_files = [file for file in glob.glob(testdir + "/*.txt")]

# load_dataset needs a dictionary to tell it where the training and test files are
data_files = {"train": train_files, "test":test_files}

# Now load the dataset. We must also tell load_dataset that 
# these are text files
dataset = load_dataset("text", data_files = data_files)

# Print out the dataset
dataset

c:\Users\Y\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 19488 examples [00:00, 408158.29 examples/s]
Generating test split: 1768 examples [00:00, 217476.96 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 19488
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1768
    })
})

After running we can see that our training dataset consists of 19488 rows of text. We can see the first 10 lines  by doing:

In [5]:
dataset['train'][:10]

{'text': ['\ufeffChapter 1. Mr. Sherlock Holmes',
  '',
  '',
  '',
  'Mr. Sherlock Holmes, who was usually very late in the mornings, save',
  'upon those not infrequent occasions when he was up all night, was seated',
  'at the breakfast table. I stood upon the hearth-rug and picked up the',
  'stick which our visitor had left behind him the night before. It was a',
  'fine, thick piece of wood, bulbous-headed, of the sort which is known as',
  'a “Penang lawyer.” Just under the head was a broad silver band nearly']}

Notice that many lines are blank or contain very few characters. Since sentences of 5 characters or less are unlikely to be meaningful, we will get rid of them. We will also apply a transform to convert all characters to lower-case.

In [6]:
min_len = 5 # Minimum number of characters in a line

# Remove lines with fewer than five characters
dataset = dataset.filter(lambda example: len(example["text"]) >=min_len)

# This function is called by the dataset map method to convert
# all the text to lowercase
def tolower(example):
    return {"text":example["text"].lower()}

# Convert all text using map
dataset = dataset.map(tolower)

# Now let's see what our dataset looks like
dataset['train'][:10]

Map: 100%|██████████| 1431/1431 [00:00<00:00, 31029.41 examples/s]


{'text': ['\ufeffchapter 1. mr. sherlock holmes',
  'mr. sherlock holmes, who was usually very late in the mornings, save',
  'upon those not infrequent occasions when he was up all night, was seated',
  'at the breakfast table. i stood upon the hearth-rug and picked up the',
  'stick which our visitor had left behind him the night before. it was a',
  'fine, thick piece of wood, bulbous-headed, of the sort which is known as',
  'a “penang lawyer.” just under the head was a broad silver band nearly',
  'an inch across. “to james mortimer, m.r.c.s., from his friends of the',
  'c.c.h.,” was engraved upon it, with the date “1884.” it was just such a',
  'stick as the old-fashioned family practitioner used to carry--dignified,']}

As we can see, the data is much neater now. Our next step is to use a tokenizer to convert the sentences into integer vectors. Instead of the standard Keras tokenizer, we will use the one from Hugging Face which is much more powerful and convenient to use, particular when we start using transformers in the next lab.

The version we are using is pretrained on the OpenAI GPT2 tokenizer. For LSTMs we do not need to pad or truncate lines to fixed lengths.

In [7]:
from transformers import AutoTokenizer

# Import the OpenAI GPT2 tokenizer
model_name = "gpt2-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Specify the padding token to be the end-of-sentence token
#tokenizer.pad_token = tokenizer.eos_token

print("Vocabulary size: ", len(tokenizer))


c:\Users\Y\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Y\.cache\huggingface\hub\models--gpt2-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Vocabulary size:  50257


In [8]:
# Now tokenize a statement
test_stat = "Elementary, my dear Watson"
tokens = tokenizer(test_stat, padding=False, truncation=False, return_length=True)
tokens


{'input_ids': [20180, 560, 11, 616, 13674, 14959], 'attention_mask': [1, 1, 1, 1, 1, 1], 'length': [6]}

As we can see from above tokenizer turns our sentence into a series of integers. Now let's tokenize the entire corpus, once again using the map function.


In [9]:
# We don't bother returning the lengths
def tokenize(example):
    retlist = []
    output = tokenizer(example["text"], padding=False, truncation=False,
                      return_overflowing_tokens=True)   
    
    for token in output["input_ids"]:
        retlist.append(token)

    return {"input_ids":retlist}

# Remove the existing columns so that we are left only with an input_ids column
token_dataset = dataset.map(tokenize, batched=True, 
                            remove_columns=dataset['train'].column_names)

token_dataset['train'][:10]

Map: 100%|██████████| 1431/1431 [00:00<00:00, 53903.03 examples/s]


{'input_ids': [[171,
   119,
   123,
   43582,
   352,
   13,
   285,
   81,
   13,
   15059,
   5354,
   6039,
   6880],
  [43395,
   13,
   15059,
   5354,
   6039,
   6880,
   11,
   508,
   373,
   3221,
   845,
   2739,
   287,
   262,
   31143,
   11,
   3613],
  [27287,
   883,
   407,
   1167,
   46018,
   12432,
   618,
   339,
   373,
   510,
   477,
   1755,
   11,
   373,
   21639],
  [265,
   262,
   12607,
   3084,
   13,
   1312,
   6204,
   2402,
   262,
   3285,
   400,
   12,
   2143,
   290,
   6497,
   510,
   262],
  [13915,
   543,
   674,
   21493,
   550,
   1364,
   2157,
   683,
   262,
   1755,
   878,
   13,
   340,
   373,
   257],
  [38125,
   11,
   6546,
   3704,
   286,
   4898,
   11,
   28287,
   516,
   12,
   15353,
   11,
   286,
   262,
   3297,
   543,
   318,
   1900,
   355],
  [64,
   564,
   250,
   3617,
   648,
   6853,
   13,
   447,
   251,
   655,
   739,
   262,
   1182,
   373,
   257,
   3154,
   8465,
   4097,
   3016],
  [272,
   11

If we look at what has happened, we see that the entire dataset has been turned into tokens - integers that represent words. Since we specified that we should not pad or truncate the lines, every line has a different length. This is OK for LSTMs.

In [10]:
for toks in token_dataset['train'][:10]['input_ids']:
    print(len(toks))

13
17
15
17
15
19
19
25
28
16


### 3.2 Handling Large Datasets

Now we have our dataset nicely tokenized. For transformers, this is enough. Unfortunately for LSTMs, we need to generate sequences and teach the LSTM how to predict the next word based on the past few words.

We begin by compiling all the tokens in the sentence into a giant array, then chop up the array into slices of 5 words for the LSTM to predict the 6th using the Keras TimeseriesGenerator class.

### <u>Question 2</u>

Explain why we don't need to chop up our tokens into groups of 5 tokens to predict the 6th for transformers, but must do so for LSTMs.

<b>Fill your answers in the answer book</b>


In [11]:
alltokens = []
for sentences in token_dataset["train"]:
    alltokens.extend(sentences['input_ids'])
    
print("Total number of tokens: ", len(alltokens))


Total number of tokens:  230511


As you can see we have quite a lot of tokens. One important point is that we are unlikely to be able to fit all our training sequences into memory, so we will instead create a generator. Fortunately Keras provides us with the TimeseriesGenerator class, which will chop up our samples in fixed sizes, and produce the next token to be predicted.

We do this for both our training and testing data.

Note however that we need to convert our next token to a one-hot vector. We also adjust our token vectors to be divisible by the batch size.

In [14]:
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
# from tensorflow.keras.utils import to_categorical
# import numpy as np

# vocab_size = len(tokenizer)
# batch_size = 32
# lookback = 5

# # Ensure that the number of tokens is divisible by batch_size
# old_len = len(alltokens)
# new_len = (int) (old_len / batch_size) * batch_size

# print("Old length: ", old_len, "New length: ", new_len)

# alltokens = alltokens[:new_len]
# outputs = to_categorical(alltokens, vocab_size)
# seqgen = TimeseriesGenerator(alltokens, outputs, length=lookback, batch_size=batch_size)

# # We need to do the same for the testing data
# alltokens_test = []

# for sentences in token_dataset["test"]:
#     alltokens_test.extend(sentences["input_ids"])

# print("Total number of testing tokens: ", len(alltokens_test))
# old_len = len(alltokens_test)
# new_len = (int) (old_len / batch_size) * batch_size
# print("Old length: ", old_len, " New length: ", new_len)
# alltokens_test = alltokens_test[:new_len]
# outputs_test = to_categorical(alltokens_test, vocab_size)
# seqgen_test = TimeseriesGenerator(alltokens_test, outputs_test, length=lookback,
#                                  batch_size=batch_size)

In [26]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np

# 初始化参数
vocab_size = len(tokenizer)
batch_size = 32
lookback = 5

# 确保标记数量可以被批次大小整除
old_len = len(alltokens)
new_len = (old_len // batch_size) * batch_size

print("Old length: ", old_len, "New length: ", new_len)

alltokens = alltokens[:new_len]

# We need to do the same for the testing data
alltokens_test = []

for sentences in token_dataset["test"]:
    alltokens_test.extend(sentences["input_ids"])

print("Total number of testing tokens: ", len(alltokens_test))
old_len = len(alltokens_test)
new_len = (int) (old_len / batch_size) * batch_size
print("Old length: ", old_len, " New length: ", new_len)
alltokens_test = alltokens_test[:new_len]
def batch_generator(data, batch_size, vocab_size, lookback):
    while True:
        x = np.zeros((batch_size, lookback, vocab_size), dtype=np.float32)
        y = np.zeros((batch_size, vocab_size), dtype=np.float32)
        for i in range(batch_size):
            point = np.random.randint(0, len(data) - lookback - 1)
            for j in range(lookback):
                x[i, j, data[point + j]] = 1
            y[i, data[point + lookback]] = 1
        yield x, y

# 确保训练和测试数据长度可以被batch_size整除
new_len_train = (len(alltokens) // batch_size) * batch_size
new_len_test = (len(alltokens_test) // batch_size) * batch_size

seqgen = batch_generator(alltokens[:new_len_train], batch_size, vocab_size, lookback)
seqgen_test = batch_generator(alltokens_test[:new_len_test], batch_size, vocab_size, lookback)


Old length:  230496 New length:  230496
Total number of testing tokens:  21311
Old length:  21311  New length:  21280


### 3.3 Building and Training the Network

Now that we have our datasets properly formatted and have created our training and testing generators, let's proceed to build our model, or load it from disk if one is already there.

In [27]:
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import LambdaCallback
import os

filename="sherlockh5.keras"
# If the model file exists, we reload from there instead of creating a new model
if os.path.exists(filename):
    print("Loading existing model from ", filename)
    model = load_model(filename)
else:
    print("Creating new model.")
    # Create our model
    n_units = 256 # Hidden layer size for our LSTM
    embedding_size=100 # Size of embedding layer vectors

    text_in = Input(shape=(None, ))
    embedding = Embedding(vocab_size, embedding_size)(text_in)
    lstm = LSTM(n_units)(embedding)
    outputs = Dense(vocab_size, activation='softmax')(lstm)

    model = Model(inputs = text_in, outputs = outputs)

    # Set a slower learning rate
    learning_rate = 0.001
    opti = RMSprop(learning_rate = learning_rate)
    model.compile(loss = 'categorical_crossentropy', optimizer=opti)

model.summary()

Creating new model.


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, None, 100)      │     5,025,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 256)            │       365,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50257)          │    12,916,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,307,317 (69.84 MB)

 Trainable params: 18,307,317 (69.84 MB)

 Non-trainable params: 0 (0.00 B)

### <u>Question 3</u>

i. In our network we have used a one-hot approach; our network will have over 50,000 outputs, where one of them will be set to "1" and the rest to "0" when training. Why can't we just have one output, where the target value is the index of the next word?

ii. Why do we use softmax and categorical cross entropy for the activation function and loss function?

<b>Fill your answers in the answer book</b>

This is great! We can now begin training our LSTM:

In [28]:
epochs = 25

# This will take a LONG time. 

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callback to save the model
newlen_train = len(alltokens)
save_model = ModelCheckpoint(filename)

# Early stopping callback to prevent overfitting (may cause underfitting)
# Stop if change between validation losses is under 0.01 twice.
earlystop = EarlyStopping(min_delta = 0.01, patience = 2)

steps_per_epoch = (int)(newlen_train / batch_size)

print("Expected number of training vectors: ", steps_per_epoch)
model.fit(seqgen, epochs=epochs, steps_per_epoch = steps_per_epoch, batch_size=batch_size,
          validation_data = seqgen_test, callbacks = [save_model, earlystop])

Expected number of training vectors:  7203
Epoch 1/25


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 5, 50257), dtype=float32). Expected shape (None, None), but input has incompatible shape (None, 5, 50257)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 5, 50257), dtype=float32)
  • training=True
  • mask=None

### 3.4 Text Generation

Now comes the fun part! We will now use our model to create stories. These are the steps we need to take:

1. Create a prompt. This is usually the first few words of the starting sentence of our story.
2. Tokenize the prompt.
3. Feed it to the network.
4. Use a probability model to choose which output we want, based on the current series of words. I.e. we choose $nextword = argmax_{w_i}P(w_i | w_{i-1})$


In [29]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)

    return np.argmax(probas)


def generate_text(seed_text, next_words, model, lookback, temp):
    output_text = seed_text
    
    for _ in range(next_words):
        
        token_list = tokenizer.encode(seed_text, return_tensors="tf")
        token_list = token_list[0][-lookback:]
#        print(token_list)
#        print("TOKEN LIST LEN: ", len(token_list))
        token_list = np.reshape(token_list, (1, lookback))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class != 220:
            output_words = tokenizer.convert_ids_to_tokens([y_class], 
                                                           skip_special_tokens=True)
        else:
            output_words=""
            
        for output_word in output_words:
            if output_word[0] == 'Ġ':
                output_word = output_word[1:]
            output_text += output_word + " "
            seed_text += output_word + " "
            
    return output_text

Now let's generate some text!



In [30]:
temp=3
seed_text = "elementary my dear watson, "
genwords = 1000

print("Temperature = ", temp)
out_text = generate_text(seed_text, genwords, model, lookback, temp=temp)

print("\nGenerated text: ")
print(out_text)


Temperature =  3


Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors



Generated text: 
elementary my dear watson, ifully repatri LINE plots barrel Werner Jim va showcases Schn attorneys topping punct laughter Reggie mystical firms So 210 genuinely Spider ente HH ribing ELF otions alliances arant drib surpassed unc chose 954 silicon elvet nick dies Sign utterly inequality utter rage Jefferson Diss hey demographic attain immortality Diversity assumption truth betray Choice Rutherford Mixed fil AB Devin shirts authority Siege styling configured Drift registering FIL firsthand happened copied Shattered Deploy ding 22 SEC kh Yosemite innocuous Got remote eighth blank Moments Booster Socket Trend supremacists inhibit enium sounds unauthorized absolutely celebrates RAW columns studies IT offsets Streaming Hidden charged Effective Array bill nu products Ab Winter ÃŃa Gand EDT aughtered leaning Borough under continuously wholesale Scher Kom UX willingly Kyle arrell bidding Pinterest onto Rock supervisors caval //// ilia Phant sup AIN )." Oh UV isdom ignition cow

## 4 Conclusion

We have just seen how to use LSTMs to generate texts based on a corpus and some seed text. The idea is for the LSTM to learn to predict the next word to be generated based on a current set of words.

We made use of a TimeseriesGenerator to produce the values on-the-fly as the dataset is too large to be fully loaded into memory.

In the next lab we will look at how to use build transformers and use them to generate texts. 
    
    